In [33]:
from pathlib import Path
from math import exp, cos, sin, pi
import numpy as np
import json
from graphix.channels import KrausChannel, KrausData, depolarising_channel, dephasing_channel



# import random
# import graphix.command
# import stim
# import veriphix.client
# from graphix.fundamentals import IXYZ
# from graphix.noise_models import DepolarisingNoiseModel

# from graphix.pauli import Pauli
# from graphix.random_objects import Circuit, rand_circuit
# from graphix.sim.density_matrix import DensityMatrixBackend
# from graphix.sim.statevec import Statevec, StatevectorBackend
# from graphix.states import BasicStates
# from veriphix.client import CircuitUtils, Client, Secrets

# import gospel.brickwork_state_transpiler
# from gospel.scripts.qasm2brickwork_state import read_qasm, draw_brickwork_state_colormap



### Data loading

In [12]:
path = Path("data/")
file = path / "NVnoise.json"

with open(file) as f:
    data = json.load(f)

print(data)

{'description': 'On Chip NVCenter with explicit electronic spin', 'ngPrep': [[[[0.6965478492097076, -0.08076076345614948], [-0.08076076345585967, 0.696547849209095]], [[-0.08076076345166208, -0.6965478491697078], [-0.6965478491704048, -0.08076076345122056]], [[0.06435326070013236, -0.06447829642850145], [0.06447829642840705, -0.0643532607002267]], [[-0.06447829638010338, -0.06435326065194914], [0.06435326065192246, 0.06447829638013021]]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]]], 'durPrep': 0.007244051177127201, 'ngCZ': [[[0.932466130909677, 0, 0, 0, 0, 0, 0, 0], [0, 0.9324661309096767, 0, 0, 0, 0, 0, 0], [0, 0, 0.932466130909677, 0, 0, 0, 0, 0], [0, 0, 0, 0.932466130909677, 0, 0, 0, 0], [0, 0, 0, 0, 0.9324661309096771, 0, 0, 0], [0, 0, 0, 0, 0, 0.9324661309096767, 0, 0], [0, 0, 0, 0, 0, 0, 0.932466130909677, 0], [0, 0, 0, 0, 0, 0, 0, 0.932466130909677]], [[0.11960508981538805, 0, 0, 0, 0, 0, 0, 0], [0, -0.11960508981538702, 0, 0, 0, 0, 0, 0], [0, 0, -0

### Preparation noise model extraction and setup

In [ ]:
## load T1 and T2 (UNITS?)
# first is electron
# nuclear spins T1/T2
from typing import Iterable


T1_list = data["T1"][1:]
T2_list = data["T2"][1:]
#print(t1_list)

## load crosstalk f (UNITS?)
f_crosstalk = data["crosstalk"]

## load preparation parameters
# Kraus operators
print(len(data["ngPrep"][0]))

# duration
prep_duration = data["durPrep"]


# normalisation checked at instanciation
# if succeeds, channel is normalised
prep_channel = KrausChannel([KrausData(1, np.array(k)) for k in data["ngPrep"][0]])
print(f"prep_channel acts on {prep_channel.nqubit} qubit and has {len(prep_channel)} Kraus operators.")


# no method to compose channels, only noise models
# shoul dbe able to tune the prep duration param. Duplicate
# assume all nuclear spins have the same T1
# this is for passive qubits
depol_channel_passive = depolarising_channel(0.75 * (1 - exp(- prep_duration/T1_list[0])))
deph_channel_passive = dephasing_channel(0.5 * (1 - exp(- prep_duration/T2_list[0])))
# TO CHECK WITH CICA: -> no noise
# depol_channel_active = depolarising_channel(0.)
# deph_channel_active = dephasing_channel(0.)

# cross talk

# ZZ_TENSOR = np.array(
#     [[[[1, 0], [0, 0]], [[0, -1], [0, 0]]], [[[0, 0], [-1, 0]], [[0, 0], [0, 1]]]],
#     dtype=np.complex128,
# )

# matrix form for use in evolve that reshapes the matrix to tensor
ZZ_MATRIX = np.array(
    [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]],
    dtype=np.complex128,
)
def exp_ZZ(theta: float) -> Iterable[complex]:
    return cos(theta) * np.eye(4, dtype=np.complex128) + 1j * sin(theta) * ZZ_MATRIX

#print(exp_ZZ(2.12)@ exp_ZZ(2.12).conj().T)

# optional: a gate is a unitary channel
exp_ZZ_channel = KrausChannel([KrausData(1, exp_ZZ(pi * prep_duration / f_crosstalk))])


4
prep_channel acts on 1 qubit and has 4 Kraus operators.


### CZ noise extraction and setup

### Measurement noise extraction and setup